In [4]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [32]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')

In [33]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
10474,Tired Hands / Omnipollo Double Milkshake IPA -...,4.0,"Ardmore, Pennsylvania",Tired Hands Brewing Company,Hazy golden color with a medium white head tha...,egajdzis
10475,Thornbridge Days of Creation,4.0,"Bakewell, Derbyshire",Thornbridge Brewery,"Bottle from 100 piv. Aroma: wooden, vanilla, r...",Barea
10476,Black Iris Mother Knows Best,3.8,"Nottingham, Nottinghamshire",Black Iris Brewery,Can from Mother Kelly’s. Slight hazy golden co...,Mr_Pink_152
10477,Dieu du Ciel! Péché Mortel (Bourbon),4.6,"Montréal, Quebec",Brasserie Dieu du Ciel!,Black pour with a long lasting dark brown head...,Listigovers


In [ ]:
geolocator = Nominatim(user_agent="None")

In [14]:
locations_df = pd.DataFrame(df['brewery_location'].unique(),columns=["locations"])

In [16]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [17]:
locations_df["latitude"] = locations_df.locations.apply(lambda x: tryconvert(x))

In [18]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [19]:
locations_df["longitude"] = locations_df.locations.apply(lambda x: tryconvertlong(x))

In [20]:
locations_df

,locations,latitude,longitude
0,"Bungay, Suffolk",52.456225,1.437341
1,"Faversham, Kent",51.314409,0.891189
2,"Bristol, Gloucestershire",51.852767,-2.255369
3,"Round Rock, Texas",30.508235,-97.678893
4,"Brooklyn, New York",40.650104,-73.949582
...,...,...,...
1378,"Vielank, Mecklenburg-Vorpommern",53.235501,11.135182
1379,"Baileys Harbor, Wisconsin",45.064993,-87.124274
1380,"Broomfield, Colorado",39.920383,-105.069146
1381,"Homerton, Greater London",51.547079,-0.043121


In [28]:
countries = []
for row in locations_df.index:
    try:
        loc = geolocator.reverse([locations_df.iloc[row,1],locations_df.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [29]:
locations_df['country'] = countries

In [31]:
locations_df.to_csv('locations_geotagged.csv',index=False)

## Deal with Non-English Descriptions